In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [ ]:
from collections import namedtuple
Sales = namedtuple("Sales",["id","account","year","commission","sales_reps"])

sales = sc.parallelize([Sales(1, "Acme", "2013", 1000, ["Jim", "Tom"]),
         Sales(2, "Lumos", "2013", 1100, ["Fred", "Ann"]),
         Sales(3, "Acme", "2014", 2800, ["Jim"]),
         Sales(4, "Lumos", "2014", 1200, ["Ann"]),
         Sales(5, "Acme", "2014", 4200, ["Fred", "Sally"])]).toDF()

sales.show()

In [ ]:
from pyspark.sql.functions import explode

sales.select("id","account","year","commission",explode("sales_reps").alias("sales_rep")).show()

In [ ]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

column_len = UserDefinedFunction(lambda x: len(x), IntegerType())

exploded = sales.select("id","account","year","commission",
             column_len("sales_reps").alias("num_reps"),
             explode("sales_reps").alias("sales_rep"))

exploded.show()

In [ ]:
exploded = exploded.withColumn("share", exploded.commission / exploded.num_reps).drop("num_reps")
exploded.show()

In [ ]:
exploded.groupBy("sales_rep").pivot("year").sum("share").orderBy("sales_rep").show()

In [ ]:
exploded.groupBy("account", "sales_rep").pivot("year").sum("share").orderBy("account", "sales_rep").show()

In [ ]:
sc.stop()